In [43]:
# Importing necessary libraries 
import os
import json
import requests
from astropy.table import Table
from urllib.parse import urlencode
from astropy.table import conf

### General Structure of Reading JSON files from SLED and FITS Imaging Downloads from Multiple Databases.

In [44]:
def load_sled_table(json_file):
    """
    Load JSON file of lensed objects into an Astropy Table.
    Each row corresponds to a unique 'name' entry in the JSON.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    rows = []
    for entry in data:
        rows.append({
            "name": entry.get("name", "Unknown"),
            "ra": float(entry.get("ra", None)),
            "dec": float(entry.get("dec", None)),
            "score": entry.get("score", None),
            "flag": entry.get("flag", "Unknown"),
            "n_img": entry.get("n_img", None),
            "image_sep": entry.get("image_sep", None),
            "lens_type": entry.get("lens_type", "Unknown"),
            "source_type": entry.get("source_type", "Unknown"),
            "imaging": entry.get("imaging", [])
        })

    table = Table(
        rows=rows,
        names=[
            "name", "ra", "dec", "score",
            "flag", "n_img", "image_sep", "lens_type", "source_type", "imaging"
        ]
    )
    return table

In [45]:
def download_fits(url, outpath):
    """
    Download a FITS file from a URL and save it locally.
    """
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    try:
        r = requests.get(url, timeout=200)
        r.raise_for_status()
        with open(outpath, "wb") as f:
            f.write(r.content)
        print(f"Saved {outpath}")
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")

In [ ]:
def query_panstarrs(ra, dec, imaging, outdir, name):
    """
    Download Pan-STARRS FITS cutouts for each band in imaging.
    """
    base = "http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
    valid_bands = {"g", "r", "i", "z", "y"}  # PS1 filters

    for img in imaging:
        instrument = img.get("instrument")
        if instrument != "Pan-STARRS1":
            continue

        band = img.get("band", "").lower()
        if band not in valid_bands:
            print(f"Skipping {name} due to invalid or missing band: {band}")
            continue

        pixel_size = img.get("pixel_size")
        if pixel_size is None or pixel_size <= 0:
            pixel_size = 120  # default arcseconds

        params = {
            "ra": ra,
            "dec": dec,
            "size": pixel_size,
            "format": "fits",
            "filter": band
        }

        url = base + urlencode(params)

        # Path structure: outdir/name/instrument/name_band.fits
        inst_dir = os.path.join(outdir, name, instrument)
        os.makedirs(inst_dir, exist_ok=True)

        outpath = os.path.join(inst_dir, f"{name}_{band}.fits")
        print(f"Downloading {name} {band} -> {outpath}")
        download_fits(url, outpath)

### Accessing JSON lensed galaxies file

### Accessing JSON lensed quasars file 

### Accessing JSON lensed supernovae file 


In [47]:
if __name__ == "__main__":
    json_path = "./lensed_supernovae/supernovae.json"
    outdir = "./lensed_supernovae/cutouts"

    # Load JSON catalog into an Astropy Table
    table = load_sled_table(json_path)
    print(table)

    # Loop through catalog entries and fetch Pan-STARRS cutouts
    for row in table:
        query_panstarrs(
        ra=row["ra"],
        dec=row["dec"],
        imaging=row["imaging"],
        outdir=outdir,
        name=row["name"]
    )

     name      ...
-------------- ...
    J0138-2155 ...
 ABELL370_GLSN ...
MACSJ1149_GLSN ...
    SN-2022qmx ...
     iPTF16geu ...
Failed to fetch http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=y: 400 Client Error: fitscut error for url: http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=y
Failed to fetch http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=r: 400 Client Error: fitscut error for url: http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=r
Failed to fetch http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=z: 400 Client Error: fitscut error for url: http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&dec=-21.9256&size=0.250&format=fits&filter=z
Failed to fetch http://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra=24.5155&d